In [1]:
import mitsuba as mi
import drjit as dr 
import matplotlib.pyplot as plt
mi.set_variant("cuda_ad_rgb")

# Surface Reconstruction based on Photometric Stereo

## Get Image $I$

In [2]:
## get image I ##
bmp = mi.Bitmap('../scenes/img/normal-10-10.png')
I_t = mi.TensorXf(bmp)/256

## Set LED Configuration

In [3]:
import utils.led_ring

N_LED = 24
LED0_pos = mi.Point3f(-34.41,0,-12.5)
LED_ring = utils.led_ring.LED_params(N_LED, LED0_pos)
P_LED = LED_ring.P
D_LED = LED_ring.D

## get LED intensity ##
A = 0.4
B = 0.5
LED_ring.set_sine_intensity(A,B)

## Set Camera Configuration

In [4]:
### Image pixel resolution ###
image_res = (256,256)
### camera origin          ###
cam_origin = mi.Point3f(0,0,-50)
### camera fov in mm       ###
cam_fov = (70,70)

from utils import camera 
cam = camera.Camera(cam_origin, image_res,cam_fov)

## Surface Reconstruction Iteration

In [5]:
from utils import iristac
import numpy as np
from utils.depth_from_grad import frankotchellappa, fast_poisson
from decimal import Decimal

sensor = iristac.sensor(cam,LED_ring)

Z = None
N_Pixel = image_res[0]*image_res[1]

### Ground truth ###
scene_ref = mi.load_file('../scenes/iristac_ply.xml')
si_ref = cam.shot(scene_ref)
Z_ref = si_ref.p[2]


for i in range(6):
    if i>0:
        sensor.set_Z0(Z_t)
    normal = sensor.get_surface_normal(I_t)
    grad_x = np.reshape(-normal[0]/normal[2],(image_res[1],image_res[0]))
    grad_y = np.reshape(-normal[1]/normal[2],(image_res[1],image_res[0]))
    Z = fast_poisson(grad_x,grad_y)
    Z = Z*np.sqrt(cam_fov[0]*cam_fov[1]/N_Pixel)
    error_rms = np.mean(np.sqrt(((np.ravel(Z)-Z_ref))**2))
    error_std = np.std((np.ravel(Z)-Z_ref)**2)
    print("fast poisson solver reconstruction error:\n\
    rms error:   \t {0:.2E}\n\
    std  error:  \t {1:.2E}\n\
    ".format(Decimal(error_rms),Decimal(error_std))
    )
    Z_t = mi.TensorXf(np.ravel(Z),(cam.image_res[0],cam.image_res[1]))


fast poisson solver reconstruction error:
    rms error:   	 1.61E-2
    std  error:  	 1.02E-2
    
fast poisson solver reconstruction error:
    rms error:   	 2.17E-2
    std  error:  	 2.17E-3
    
fast poisson solver reconstruction error:
    rms error:   	 1.11E-2
    std  error:  	 5.19E-4
    
fast poisson solver reconstruction error:
    rms error:   	 1.20E-2
    std  error:  	 5.98E-4
    
fast poisson solver reconstruction error:
    rms error:   	 1.20E-2
    std  error:  	 5.90E-4
    
fast poisson solver reconstruction error:
    rms error:   	 1.20E-2
    std  error:  	 5.91E-4
    
